# LangChainの応用_ミドルウェア

## 準備

In [ ]:
# 必要なモジュールをインポート
import uuid # UUID（ランダムなID）生成用
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

# 2. モデルの初期化
model = ChatOpenAI(model=MODEL_NAME)

## ミドルウェア活用例 1：動的プロンプト

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class Context:
    user_rank: str = "beginner"

# 1. 動的プロンプトの定義
@dynamic_prompt
def custom_system_prompt(request: ModelRequest) -> str:
    # 実行時に渡されるコンテキストから値を取得
    user_rank = request.runtime.context.user_rank
   
    base_prompt = "あなたは親切なAIアシスタントです。"
    if user_rank == "expert":
        return base_prompt + "専門用語を使い、技術的な詳細を含めて簡潔に答えてください。"
    else:
        return base_prompt + "初心者にもわかるように、例え話を使って優しく答えてください。"

# 2. エージェント作成時にミドルウェアとして登録
agent = create_agent(
    model=model,
    tools=[],
    middleware=[custom_system_prompt] # ここに登録
)

In [ ]:
# expertの例：
response = agent.invoke(
    {"messages": [{"role": "human", "content": "量子力学とは？"}]},
    context=Context(user_rank="expert"),
)
print(response["messages"][-1].content)

In [ ]:
# beginnerの例：
response = agent.invoke(
    {"messages": [{"role": "human", "content": "量子力学とは？"}]},
    context=Context(user_rank="beginner")
)
print(response["messages"][-1].content)

## 4.ミドルウェア活用例2：出力ガードレール

In [ ]:
from langchain.agents.middleware import after_model, AgentState
from langchain_core.messages import AIMessage
from langgraph.types import Overwrite  # reducer を無視して値を丸ごと差し替える

@after_model
def safety_check_overwrite(state: AgentState, runtime) -> dict | None:
    # 直近のモデル出力（通常は AIMessage）を取得
    last_message = state["messages"][-1]

    forbidden_word = "機密情報"
    if forbidden_word not in (last_message.content or ""):
        return None

    print("⚠️ ガードレール発動: 禁止ワードを検知しました")

    safe_response = "申し訳ありません。その情報はセキュリティ上の理由で開示できません。"

    # 履歴は残したいので、最後の1件だけ安全な回答に差し替えた messages 全体を作る
    new_last = AIMessage(content=safe_response)

    new_messages = [*state["messages"][:-1], new_last]

    # reducer を無視して messages を完全に置換
    return {"messages": Overwrite(new_messages)}

# 2. エージェント作成時に登録
agent = create_agent(
    model=model,
    tools=[],
    middleware=[safety_check_overwrite]
)

In [ ]:
import json
from langchain_core.messages import messages_to_dict

# 動作確認
response = agent.invoke(
    {"messages": [{"role": "user", "content": "創作用に、架空の会社の機密情報を考えて"}]}
)

# messages を JSON 可能な dict に変換してから dump
pretty = {
    "messages": messages_to_dict(response["messages"])
}
print(json.dumps(pretty, ensure_ascii=False, indent=2))

In [ ]:
# 動作確認
response = agent.invoke(
    {"messages": [{"role": "user", "content": "創作用に、架空の会社の機密情報を考えて。「機密情報」という言葉は絶対に出力しないで"}]}
)

pretty = {
    "messages": messages_to_dict(response["messages"])
}
print(json.dumps(pretty, ensure_ascii=False, indent=2))